# Copy and Update Dashboards using Arize GraphQL API

### Objective

The objective of this notebook is to facilitate the easy duplication of an existing dashboard configuration and adapt it to a new model environment. This includes:
- Fetching the existing dashboard configuration.
- Modifying and updating dashboard widgets to suit the new model.
- Saving the updated configuration to the Arize platform.

### Step 1: Initialize the GraphQL Client using your developer API key

In [ ]:
!pip install -q gql[all]
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import time

### Get your API key
First, make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), then you have developer permissions. If not, please ask your Account Admin to provide you with access.

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below.

NOTE: this key is different than the SDK key used to send data to Arize.

In [ ]:
API_KEY = ""

# Select your transport with a defined URL endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

## Get the target dashboard information

Query to retreive all the widgets from the dashbaord you wish to copy.


In [ ]:
# This query works at the model level
# The url will be in this format: https://app.arize.com/organizations/:orgId/spaces/:spaceId/models/:modelId

DASHBOARD_ID = ""


dashboard_query = gql(
    """
query getDashboard($dashboardId: ID!) {
  node(id: $dashboardId) {
    ... on Dashboard {
      id
      driftLineChartWidgets {
        edges {
          node {
            title
            gridPosition
            timeSeriesMetricType
            creationStatus
            yMin
            yMax
            plots {
              title
              position
              modelId
              dimensionCategory
              metric
              customMetric {
                id
              }
              splitByEnabled
              splitByDimension
              splitByDimensionCategory
              cohorts
              dimension {
                id
              }
              predictionValueClass {
                id
              }
              rankingAtK
              filters {
                id
                filterType
                operator
                dimension {
                  id
                  name
                  dataType
                }
                dimensionValues {
                  id
                  value
                }
                binaryValues
                numericValues
                categoricalValues
              }
              modelPrimaryBaseline {
                referenceType
                filteredBaseline {
                  modelEnvironmentName
                  filteredMovingWindowSeconds
                  filteredMovingWindowDelaySeconds
                  filteredStartDate
                  filteredEndDate
                }
                modelVersionEnvironmentBatch {
                  id
                }
                baselineType
                startDateOverride
                endDateOverride
              }
              comparisonDataset {
                referenceSource
                referenceBaselineId
              }
            }
          }
        }
      }
      statisticWidgets {
        edges {
          node {
            title
            dimensionCategory
            performanceMetric
            aggregation
            predictionValueClass {
              id
              name
            }
            rankingAtK
            gridPosition
            timeSeriesMetricType
            id
            creationStatus
            filters {
              id
              filterType
              operator
              dimension {
                id
                name
                dataType
              }
              dimensionValues {
                id
                value
              }
              binaryValues
              numericValues
              categoricalValues
            }
            dimension {
              id
              name
              dataType
            }
          }
        }
      }
      barChartWidgets {
        edges {
          node {
            title
            gridPosition
            creationStatus
            sortOrder
            yMin
            yMax
            isNormalized
            binOption
            numBins
            customBins
            quantiles
            performanceMetric
            plots {
              title
              position
              modelId
              dimension {
                id
                name
                dataType
              }
              dimensionCategory
              aggregation
              rankingAtK
              filters {
                id
                filterType
                operator
                dimension {
                  id
                  name
                  dataType
                }
                dimensionValues {
                  id
                  value
                }
                binaryValues
                numericValues
                categoricalValues
              }
            }
          }
        }
      }
      lineChartWidgets {
        edges {
          node {
            title
            gridPosition
            timeSeriesMetricType
            creationStatus
            yMin
            yMax
            plots {
              title
              position
              modelId
              dimensionCategory
              metric
              customMetric {
                id
              }
              splitByEnabled
              splitByDimension
              splitByDimensionCategory
              cohorts
              dimension {
                id
              }
              predictionValueClass {
                id
              }
              rankingAtK
              filters {
                id
                filterType
                operator
                dimension {
                  id
                  name
                  dataType
                }
                dimensionValues {
                  id
                  value
                }
                binaryValues
                numericValues
                categoricalValues
              }
              modelPrimaryBaseline {
                referenceType
                filteredBaseline {
                  modelEnvironmentName
                  filteredMovingWindowSeconds
                  filteredMovingWindowDelaySeconds
                  filteredStartDate
                  filteredEndDate
                }
                modelVersionEnvironmentBatch {
                  id
                }
                baselineType
                startDateOverride
                endDateOverride
              }
              comparisonDataset {
                referenceSource
                referenceBaselineId
              }
            }
          }
        }
      }
      monitorLineChartWidgets {
        edges {
          node {
            title
            gridPosition
            timeSeriesMetricType
            creationStatus
            yMin
            yMax
            monitor {
              __typename
              ... on WidgetMonitor {
                id
                name
              }
            }
          }
        }
      }
    }
  }
}


  """
)


# Base query parameters for fetching monitors
params = {"dashboardId": str(DASHBOARD_ID)}

paged_response = client.execute(dashboard_query, params)

### Seperate each widget type to use later

In [ ]:
stats = paged_response["node"]["statisticWidgets"]
stats_df = pd.json_normalize(stats["edges"], sep=".")

In [ ]:
linecharts = paged_response["node"]["lineChartWidgets"]
linechart_df = pd.json_normalize(linecharts["edges"], sep=".")

In [ ]:
barcharts = paged_response["node"]["barChartWidgets"]
barcharts_df = pd.json_normalize(barcharts["edges"], sep=".")

In [ ]:
monitors = paged_response["node"]["monitorLineChartWidgets"]
monitors_df = pd.json_normalize(monitors["edges"], sep=".")

In [ ]:
drift = paged_response["node"]["driftLineChartWidgets"]
drift_df = pd.json_normalize(drift["edges"], sep=".")

# Create New Jobs

## Get New Model Metadata

Retrieve the new model's environment ID for later use.

In [ ]:
NEW_MODEL_ID = ""
NEW_MODEL_ENV = "production"

In [ ]:
model_query = gql(
    """
  query getModelMetaData($modelID:ID!){
  node(id: $modelID) {
    ... on Model {
      name
      modelType
      modelEnvironments{
        environmentId
        name
      }
      uri
    }
  }
}
    """
)


# Base query parameters for fetching monitors
params = {"modelID": str(NEW_MODEL_ID)}

# An array of monitors that we will append to
metadata = []
model_name = ""


paged_response = client.execute(model_query, params)

print(paged_response)

In [ ]:
df = pd.json_normalize(paged_response, record_path=None)

df = df.explode("node.modelEnvironments").reset_index(drop=True)


model_env_df = pd.json_normalize(df["node.modelEnvironments"])

result_df = pd.concat(
    [df.drop(columns=["node.modelEnvironments"]), model_env_df], axis=1
)

In [ ]:
# Assuming 'name' matches 'NEW_MODEL_ENV' and there's exactly one such match
environmentId_value = result_df.loc[
    result_df["name"] == NEW_MODEL_ENV, "environmentId"
].item()
environmentId_value

## Get features

Fetch and update features including their IDs for the new model's dashboard configuration.

In [ ]:
features_query = gql(
    """
    query getFeatures($modelId: ID!, $cursor: String) {
        model: node(id: $modelId) {
            ... on Model {
                name
              	id
              	modelEnvironments{
                  environmentId
                  name
                }
                modelSchema {
                    features(first: 20, after: $cursor, filter: { exclude:{ dataTypes:EMBEDDING }}) {
                        edges {
                            feature: node {
                                dimension {
                                  id
                                    name,
                                    dataType,
                                  category
                                }
                            }
                        }
                        pageInfo {
                            endCursor
                        }
                    }
                }
            }
        }
    }

"""
)

# Base query parameters for fetching features
params = {"modelId": NEW_MODEL_ID, "cursor": None}
# An array of features that we will append to
feature_data = []
model_name = ""

# Execute the query on the transport. Continue to pull data until there is no more features
while True:
    paged_response = client.execute(features_query, params)
    model_name = paged_response["model"]["name"]
    # Append the monitors to your list
    feature_data.extend(
        paged_response["model"]["modelSchema"]["features"]["edges"]
    )
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["model"]["modelSchema"]["features"]["pageInfo"][
        "endCursor"
    ]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of features. Loading more.")
        params["cursor"] = end_cursor
    else:
        print("No more features to pull. The list is complete!")
        break
    time.sleep(1)

print("Retrieved {} features".format(len(feature_data)))

In [ ]:
features = pd.json_normalize(feature_data, sep=".")
features.head()

## Create New Dashboard

When creating a new custom dashboard, the first step is to create an empty state dashboard using the `createDashboard` mutation. This mutation provides you with a `dashboardId` which will be used subsequently when adding widgets.

In [ ]:
create_mutation = gql("""
mutation createDashTest($name: String!, $spaceId: ID!) {
  createDashboard(input: { name: $name, spaceId: $spaceId }) {
    dashboard {
      id
    }
  }
}
""")

In [ ]:
new_dashboard_name = "Test Dashboard"
SPACE_ID = ""
params = {"name": new_dashboard_name, "spaceId": SPACE_ID}
dashboard_response = client.execute(create_mutation, params)

In [ ]:
DASHBOARD_ID = dashboard_response["createDashboard"]["dashboard"]["id"]

## Adding Distribution Widgets

This section explains how to modify the distribution widgets to align with the new model's data dimensions. Depending on whether your models share the same feature space, you may choose to either retain the existing features or update them using the feature IDs retrieved from the previous queries.

### Example: Updating Feature IDs in Widgets
Here’s how you can update the feature IDs for your distribution widgets using the retrieved feature data. This example assumes you have a dictionary of updates ready to apply based on feature names.

In [ ]:
bar_chart_updates = {
    "feature_name": {
        "feature": "delinq_2yrs",
        "title": "distribution for delinq_2yrs",
    },
    "feature_name_2": {"feature": "state", "title": "distribution for state"},
    "state": {"feature": "dti", "title": "distribution of dti"},
}

In [ ]:
createBarChartWidgetMutation = gql("""

mutation createBarChartWidget(
$title:String!
  $creationStatus: WidgetCreationStatus!
  $dashboardId:ID!
  $gridPosition:[Int!]!
  $performanceMetric: PerformanceMetric
  $isNormalized:Boolean!
  $plots:[BarChartPlotInputInput!]!


){
  createBarChartWidget(
    input:
  {
    title:$title
    creationStatus:$creationStatus
    performanceMetric: $performanceMetric
    dashboardId: $dashboardId # returned from createDashboard
    #how can we automatically get the min available grid position?
    gridPosition: $gridPosition
    isNormalized: $isNormalized # true or false; unless user explictly says it default to False
    plots: $plots
  }
  ){
    barChartWidget{
      id

    }
  }
}
""")

In [ ]:
for id, existing_row in barcharts_df.iterrows():
    title = existing_row["node.title"]
    gridPosition = existing_row["node.gridPosition"]
    creationStatus = existing_row["node.creationStatus"]
    isNormalized = existing_row["node.isNormalized"]
    performanceMetric = existing_row["node.performanceMetric"]
    plots = existing_row["node.plots"]
    for plot in plots:
        plot["modelId"] = NEW_MODEL_ID
        plot["modelVersionIds"] = []
        plot["modelVersionEnvironmentMetadataIds"] = []
        if plot["title"] in bar_chart_updates:
            new_dim_name = bar_chart_updates[plot["title"]]["feature"]
            new_plot_title = bar_chart_updates[plot["title"]]["title"]
            new_dimension = features.loc[
                features["feature.dimension.name"] == new_dim_name
            ].iloc[0]
            plot["dimension"]["id"] = new_dimension["feature.dimension.id"]
            plot["dimension"]["name"] = new_dimension["feature.dimension.name"]
            plot["dimension"]["dataType"] = new_dimension[
                "feature.dimension.dataType"
            ]

    params = {
        "title": title,
        "gridPosition": gridPosition,
        "creationStatus": creationStatus,
        "dashboardId": (DASHBOARD_ID),
        "performanceMetric": performanceMetric,
        "isNormalized": isNormalized,
        "plots": plots,
    }
    result = client.execute(createBarChartWidgetMutation, params)
    # print(f'Widget created: {title}')
    """
      give a bit of breathing room between updates
    """
    time.sleep(0.1)

## Adding Time Series Widgets

This section details how to update the parameters for line chart widgets when adapting them to a new model within the dashboard. You have the option to retain existing metrics and dimensions or modify them to better align with the new model's data structure.

### Example: Updating Line Chart Parameters
Below are examples of how to update the parameters for two different line chart widgets. In the first widget, we update only the model ID and environment name, while in the second widget, we change the metric to falseNegativeRate instead of mae.

In [ ]:
line_chart_updates = {
    "MAE by State": {
        "plots": {
            "title": "test mae",
            "modelId": NEW_MODEL_ID,
            "modelVersionIds": [],
            "dimensionCategory": "predictionClass",
            "metric": "mae",
            "position": 1,
            "modelEnvironmentName": NEW_MODEL_ENV,
            "modelVersionIds": [],
            "filters": [],
        },
        "title": "MAE",
    },
    "MAE by home_ownership": {
        "plots": {
            "title": "test fnr",
            "position": 0,
            "modelId": NEW_MODEL_ID,
            "dimensionCategory": "predictionClass",
            "metric": "falseNegativeRate",
            "modelId": NEW_MODEL_ID,
            "modelEnvironmentName": NEW_MODEL_ENV,
            "modelVersionIds": [],
            "filters": [],
        },
        "title": "predictionClass",
    },
}

In [ ]:
createLineChartWidgetMutation = gql("""
mutation createLineChartWidget(
  $title:String!
  $dashboardId:ID!
  $gridPosition: [Int!]
  $plots:[LineChartPlotInputInput!]!
  $timeSeriesMetricType: TimeSeriesMetricCategory!
){
  createLineChartWidget(
    input:
  {
    gridPosition: $gridPosition
    title:$title
    timeSeriesMetricType: $timeSeriesMetricType
    dashboardId: $dashboardId
    plots: $plots
    widgetType: lineChartWidget
  }
  ){
    lineChartWidget{
      id
    }
  }
}
""")

In [ ]:
for id, existing_row in linechart_df.iterrows():
    title = existing_row["node.title"]
    gridPosition = existing_row["node.gridPosition"]
    timeSeriesMetricType = existing_row["node.timeSeriesMetricType"]
    if title in line_chart_updates:
        new_params = line_chart_updates[title]
        plots = new_params["plots"]
        widget_title = new_params["title"]
        params = {
            "title": widget_title,
            "gridPosition": gridPosition,
            "dashboardId": str(DASHBOARD_ID),
            "plots": [plots],
            "timeSeriesMetricType": timeSeriesMetricType,
        }
        result = client.execute(createLineChartWidgetMutation, params)
        print(f"Widget created: {title}")
    """
      give a bit of breathing room between updates
    """

    time.sleep(0.1)

## Adding Drift Widgets

In this section, we will focus on adding drift widgets to your dashboard, which involves retrieving a baseline ID and updating the widget parameters to suit the new model's metrics and dimensions.


### Retrieve a Baseline ID
Before creating drift widgets, you must retrieve a baseline ID from your existing model.

In [ ]:
getModelBaselineMutation = gql(
    """
query getBaseline($modelID:ID!){
  node(id: $modelID) {
    ... on Model {
      modelPrimaryBaseline{
        id
      }
    }
  }
}
    """
)

baseline_response = client.execute(
    getModelBaselineMutation, {"modelID": NEW_MODEL_ID}
)
BASELINE_ID = baseline_response["node"]["modelPrimaryBaseline"]["id"]

### Example: Updating Drift Widget Parameters
Once you have the baseline ID, you can proceed to update the drift widget parameters to reflect the metrics and dimensions appropriate for the new model. This section provides an example of how to keep the drift metric and dimension unchanged while updating the model_id and baseline_id to reflect the new model settings.

In [ ]:
drift_chart_updates = {
    "drift test": {
        "plots": [
            {
                "title": "test drift",
                "position": 0,
                "modelId": NEW_MODEL_ID,
                "dimensionCategory": "predictionClass",
                "metric": "psi",
                "filters": [],
                "modelVersionIds": [],
                "modelVersionEnvironmentMetadataIds": [],
                "modelEnvironmentName": "production",
                "comparisonDatasetModelBaselineId": BASELINE_ID,
            }
        ],
        "title": "Prediction Drift",
    }
}

In [ ]:
createDriftWidgetMutation = gql(
    """
    mutation createDriftWidget(
    $title:String!
      $dashboardId:ID!
      $plots:[LineChartPlotInputInput!]!
      $timeSeriesMetricType: TimeSeriesMetricCategory!
    ){
      createLineChartWidget(
        input:
      {
        title:$title
        timeSeriesMetricType: $timeSeriesMetricType
        dashboardId: $dashboardId
        plots: $plots
        widgetType: driftLineChartWidget
      }
      ){
        lineChartWidget{
          id
        }
      }
    }
    """
)

In [ ]:
for id, existing_row in drift_df.iterrows():
    title = existing_row["node.title"]
    gridPosition = existing_row["node.gridPosition"]
    creationStatus = existing_row["node.creationStatus"]
    plots = existing_row["node.plots"]
    # print(json.dumps(plot, indent=4))
    new_plots = {}
    if title in line_chart_updates:
        new_params = drift_chart_updates[title]
        new_plot = new_params["plots"]
        widget_title = new_params["title"]
        params = {
            "title": title,
            "gridPosition": gridPosition,
            "creationStatus": creationStatus,
            "dashboardId": (DASHBOARD_ID),
            "plots": new_plots,
        }
        result = client.execute(createDriftWidgetMutation, params)
        # print(f'Widget created: {title}')
        """
        give a bit of breathing room between updates
      """
        time.sleep(0.1)